In [159]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [160]:
df = pd.read_excel("../datasets/Student_Mobility_2013-14.xlsx")
df = df[df.MobilityType == "Mob-SMS"]

In [161]:
df = df[["SendingCountry", "ReceivingCountry", "LevelOfStudy", "MobiilityID"]]

In [175]:
receive = df.groupby(by=["ReceivingCountry", "SendingCountry", "LevelOfStudy"]).count().unstack().fillna(0)
receive = receive.reset_index(level=1)
receive.columns = ["SendingCountry", "First Cycle", "Second Cycle", "Short Cycle", "Third Cycle"]
receive.head()

,SendingCountry,First Cycle,Second Cycle,Short Cycle,Third Cycle
ReceivingCountry,,,,,
AT,BE,81.0,71.0,0.0,0.0
AT,BG,39.0,21.0,0.0,4.0
AT,CH,78.0,41.0,0.0,1.0
AT,CY,9.0,0.0,0.0,0.0
AT,CZ,83.0,198.0,0.0,9.0


In [174]:
send = df.groupby(by=["SendingCountry", "ReceivingCountry", "LevelOfStudy"]).count().unstack().fillna(0)
send = send.reset_index(level=1)
send.columns = ["ReceivingCountry", "First Cycle", "Second Cycle", "Short Cycle", "Third Cycle"]
send.head()

,ReceivingCountry,First Cycle,Second Cycle,Short Cycle,Third Cycle
SendingCountry,,,,,
AT,BE,57.0,45.0,0.0,0.0
AT,BG,2.0,1.0,0.0,0.0
AT,CH,54.0,55.0,0.0,1.0
AT,CY,2.0,4.0,0.0,0.0
AT,CZ,42.0,35.0,0.0,1.0


In [164]:
countries = ['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IS', 'IT', 'LI', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'TR']
print(len(countries))

# incoming
template = {'nodes': [], 'links': []}

33


In [165]:
result_dict = {country: {'source': [], 'target': [], 'value': []} for country in countries}
template = {'nodes': [], 'links': []}
template2 = {'source': [], 'target': [], 'value': []}

In [166]:
#sending country er source target er degree

for country in countries:
    data = receive.loc[country]
    for index, row in data.iterrows():
        result_dict[country]['source'].append(row['SendingCountry'])
        result_dict[country]['target'].append('Bachelor')
        result_dict[country]['value'].append(int(row['First Cycle']))
        result_dict[country]['source'].append(row['SendingCountry'])
        result_dict[country]['target'].append('Master')
        result_dict[country]['value'].append(int(row['Second Cycle']))
        result_dict[country]['source'].append(row['SendingCountry'])
        result_dict[country]['target'].append('PhD')
        result_dict[country]['value'].append(int(row['Third Cycle'])) 

print("HEEEEI:")
print(result_dict)

HEEEEI:
{'AT': {'source': ['BE', 'BE', 'BE', 'BG', 'BG', 'BG', 'CH', 'CH', 'CH', 'CY', 'CY', 'CY', 'CZ', 'CZ', 'CZ', 'DE', 'DE', 'DE', 'DK', 'DK', 'DK', 'EE', 'EE', 'EE', 'ES', 'ES', 'ES', 'FI', 'FI', 'FI', 'FR', 'FR', 'FR', 'GB', 'GB', 'GB', 'GR', 'GR', 'GR', 'HR', 'HR', 'HR', 'HU', 'HU', 'HU', 'IE', 'IE', 'IE', 'IS', 'IS', 'IS', 'IT', 'IT', 'IT', 'LT', 'LT', 'LT', 'LU', 'LU', 'LU', 'LV', 'LV', 'LV', 'MK', 'MK', 'MK', 'NL', 'NL', 'NL', 'NO', 'NO', 'NO', 'PL', 'PL', 'PL', 'PT', 'PT', 'PT', 'RO', 'RO', 'RO', 'SE', 'SE', 'SE', 'SI', 'SI', 'SI', 'SK', 'SK', 'SK', 'TR', 'TR', 'TR'], 'target': ['Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD

In [184]:
json_dict = {country: {"nodes": [], "links": []} for country in countries}

for key in result_dict.keys():
    datafr = pd.DataFrame.from_dict(result_dict[key])
    source_values = list(datafr.source.unique())
    count = 0
    for source in source_values:
        json_dict[key]["nodes"].append({"node": count, "name": source})
        count+=1
    json_dict[key]["nodes"].append({"node": count, "name": "Bachelor"})
    json_dict[key]["nodes"].append({"node": count+1, "name": "Master"})
    json_dict[key]["nodes"].append({"node": count+2, "name": "PhD"})
    for index, row in datafr.iterrows():
        source_nr = source_values.index(row["source"])
        if (row["target"]=="Bachelor"):
            if (row["value"] != 0):
                json_dict[key]["links"].append({"source": source_nr,"target":count,"value": row["value"]})
        elif (row["target"]=="Master"):
            if (row["value"] != 0):
                json_dict[key]["links"].append({"source": source_nr,"target":count+1,"value": row["value"]})
        elif (row["target"]=="PhD"):
            if (row["value"] != 0):
                json_dict[key]["links"].append({"source": source_nr,"target":count+2,"value": row["value"]})


In [185]:
pd.DataFrame.from_dict(result_dict['AT']).head()

,source,target,value
0,BE,Bachelor,81
1,BE,Master,71
2,BE,PhD,0
3,BG,Bachelor,39
4,BG,Master,21


In [186]:
import json

In [187]:
with open('receiving_countryselection_degree.json', 'w') as fp:
    json.dump(json_dict, fp)

### Sending
EX: dataset shows what countries norways sends to - where does norweigian students go?

In [176]:
sending_result_dict ={country: {'source': [], 'target': [], 'value': []} for country in countries}

for country in countries:
    data = send.loc[country]
    for index, row in data.iterrows():
        sending_result_dict[country]['source'].append(row['ReceivingCountry'])
        sending_result_dict[country]['target'].append('Bachelor')
        sending_result_dict[country]['value'].append(int(row['First Cycle']))
        sending_result_dict[country]['source'].append(row['ReceivingCountry'])
        sending_result_dict[country]['target'].append('Master')
        sending_result_dict[country]['value'].append(int(row['Second Cycle']))
        sending_result_dict[country]['source'].append(row['ReceivingCountry'])
        sending_result_dict[country]['target'].append('PhD')
        sending_result_dict[country]['value'].append(int(row['Third Cycle'])) 


In [181]:
sending_json_dict = {country: {"nodes": [], "links": []} for country in countries}

for key in sending_result_dict.keys():
    datafr = pd.DataFrame.from_dict(sending_result_dict[key])
    source_values = list(datafr.source.unique())
    count = 0
    for source in source_values:
        sending_json_dict[key]["nodes"].append({"node": count, "name": source})
        count+=1
    sending_json_dict[key]["nodes"].append({"node": count, "name": "Bachelor"})
    sending_json_dict[key]["nodes"].append({"node": count+1, "name": "Master"})
    sending_json_dict[key]["nodes"].append({"node": count+2, "name": "PhD"})
    for index, row in datafr.iterrows():
        source_nr = source_values.index(row["source"])
        if (row["target"]=="Bachelor"):
            if (row["value"] != 0):
                sending_json_dict[key]["links"].append({"source": source_nr,"target":count,"value": row["value"]})
        elif (row["target"]=="Master"):
            if (row["value"] != 0):
                sending_json_dict[key]["links"].append({"source": source_nr,"target":count+1,"value": row["value"]})
        elif (row["target"]=="PhD"):
            if (row["value"] != 0):
                sending_json_dict[key]["links"].append({"source": source_nr,"target":count+2,"value": row["value"]})


In [182]:
print(sending_json_dict)

{'AT': {'nodes': [{'node': 0, 'name': 'BE'}, {'node': 1, 'name': 'BG'}, {'node': 2, 'name': 'CH'}, {'node': 3, 'name': 'CY'}, {'node': 4, 'name': 'CZ'}, {'node': 5, 'name': 'DE'}, {'node': 6, 'name': 'DK'}, {'node': 7, 'name': 'EE'}, {'node': 8, 'name': 'ES'}, {'node': 9, 'name': 'FI'}, {'node': 10, 'name': 'FR'}, {'node': 11, 'name': 'GB'}, {'node': 12, 'name': 'GR'}, {'node': 13, 'name': 'HR'}, {'node': 14, 'name': 'HU'}, {'node': 15, 'name': 'IE'}, {'node': 16, 'name': 'IS'}, {'node': 17, 'name': 'IT'}, {'node': 18, 'name': 'LT'}, {'node': 19, 'name': 'LV'}, {'node': 20, 'name': 'MT'}, {'node': 21, 'name': 'NL'}, {'node': 22, 'name': 'NO'}, {'node': 23, 'name': 'PL'}, {'node': 24, 'name': 'PT'}, {'node': 25, 'name': 'RO'}, {'node': 26, 'name': 'SE'}, {'node': 27, 'name': 'SI'}, {'node': 28, 'name': 'SK'}, {'node': 29, 'name': 'TR'}, {'node': 30, 'name': 'Bachelor'}, {'node': 31, 'name': 'Master'}, {'node': 32, 'name': 'PhD'}], 'links': [{'source': 0, 'target': 30, 'value': 57}, {'so

In [183]:
with open('sending_countryselection_degree.json', 'w') as fp:
    json.dump(sending_json_dict, fp)